In [1]:
import Symbolics;
using LinearAlgebra;
using Symbolics;

In [2]:
#############################
## Variables
#############################

# Number of bodies
nbody = 2; 

# Constants
@variables G;

# Arrays for position, momentum, and masses
@variables q[1:nbody, 1:3] p[1:nbody, 1:3] m[1:nbody];

#############################
## Functions
#############################

# Obtain position vector for a body
function qx(a::Integer)
    return [q[a,1], q[a,2], q[a,3]]
end;

# Obtain momentum vector for a body
function px(a::Integer)
    return [p[a,1], p[a,2], p[a,3]]
end;

# Normalize a vector
function normal(x)
    val = norm(x)
    return [x[1] / val, x[2] / val, x[3] / val]
end;

# Obtain distance vector from one body to another
function R_ab(a::Integer, b::Integer)
    return qx(a) - qx(b)
end;

# Obtain magnitude of distance between two bodies
function R(a::Integer, b::Integer)
    return norm(R_ab(a,b))
end;

# Obtain direction of distance vector from one body to another
function n_hat(a::Integer, b::Integer)
    return normal(R_ab(a,b))
end;

# Obtain magnitude of momentum of a body
function psq(a::Integer)
    return px(a)' * px(a)
end;

# Find kinetic energy of a body
function m_line(a::Integer)
    return sqrt((m[a]^2) + psq(a))
end;

# Define commonly reused quantities for Hamiltonian potential
function y(a::Integer, b::Integer)
    return sqrt((m[a]^2) + ((px(a)' * n_hat(a,b))^2)) / (m_line(a))
end; 

function pab(a::Integer, b::Integer)
    return px(a)' * px(b)
end;

function pa_n(a::Integer, b::Integer)
    return px(a)' * n_hat(b,a)
end;

function pb_n(a::Integer, b::Integer)
    return px(b)' * n_hat(b,a)
end;

# Define functions to break up Hamiltonian equations for better readability
function fun4_1(a::Integer, b::Integer)
    return (1 / R(a,b)) * (1 / (m_line(a) * m_line(b) * ((y(b,a) + 1)^2) * y(b,a)))
end;

function fun4_2(a::Integer, b::Integer)
    line_1 = 2 * (pab(a,b)^2) * (pb_n(a,b)^2)
    line_2 = 2 * pa_n(a,b) * pb_n(a,b) * pab(a,b) * psq(b)
    line_3 = (pa_n(a,b)^2) * (psq(b)^2)
    line_4 = (pab(a,b)^2) * psq(b)
    return 2 * (line_1 - line_2 + line_3 - line_4) / (m_line(b)^2)
end;

function fun4_3(a::Integer, b::Integer)
    line_1 = psq(a) * (pb_n(a,b)^2)
    line_2 = (pa_n(a,b)^2) * (pb_n(a,b)^2)
    line_3 = 2 * pa_n(a,b) * pb_n(a,b) * pab(a,b)
    line_4 = pab(a,b)^2
    line_5 = (pa_n(a,b)^2) * psq(b)
    return 2 * (-line_1 + line_2 + line_3 + line_4 - line_5)
end;

function fun4_4(a::Integer, b::Integer)
    line_1 = 3 * psq(a) * (pb_n(a,b)^2)
    line_2 = (pa_n(a,b)^2) * (pb_n(a,b)^2)
    line_3 = 8 * pa_n(a,b) * pb_n(a,b) * pab(a,b)
    line_4 = psq(a) * psq(b)
    line_5 = 3 * (pa_n(a,b)^2) * psq(b)
    return y(b,a) * (-line_1 + line_2 + line_3 + line_4 - line_5)
end;


In [3]:
#############################
## First order Post-Minkowski Hamiltonian
#############################

# Divide equation into four sections for clarity
eq1 = sum(m_line(a) for a in 1:nbody);

eq2 = -0.5 * G * sum(
    if a == b
        0
    else
        (m_line(a) * m_line(b)) / R(a,b) 
    end
    for a in 1:nbody, b in 1:nbody
);

eq3 = 0.25 * G * sum(
    if a == b
        0
    else
        (7 * pab(a,b) + (px(a)' * n_hat(a,b)) * (px(b)' * n_hat(a,b))) / R(a,b)
    end
    for a in 1:nbody, b in 1:nbody
);

eq4 = -0.25 * G * sum(
    if a == b
        0
    else
        fun4_1(a,b) * (fun4_2(a,b) + fun4_3(a,b) + fun4_4(a,b))
    end
    for a in 1:nbody, b in 1:nbody
);

# Combine expressions to create full Hamiltonian
PM1 = eq1 + eq2 + eq3 + eq4;


In [34]:
#############################
## Generate equations of motion
#############################

# Functions for creating differentials
function Dq(a::Integer)
    return [Differential(q[a,1]), Differential(q[a,2]), Differential(q[a,3])]
end;

function Dp(a::Integer)
    return [Differential(p[a,1]), Differential(p[a,2]), Differential(p[a,3])]
end;

# Apply functions to create arrays of the equations
q_dot = Array{Num, 2}(UndefInitializer(), 2, 3)
p_dot = Array{Num, 2}(UndefInitializer(), 2, 3)

for i in 1:nbody
    q_dot[i,:] = [expand_derivatives(Dp(i)[1](PM1)), expand_derivatives(Dp(i)[2](PM1)), expand_derivatives(Dp(i)[3](PM1))]
    p_dot[i,:] = [expand_derivatives(Dq(i)[1](-PM1)), expand_derivatives(Dq(i)[2](-PM1)), expand_derivatives(Dq(i)[3](-PM1))]
end;